### Using Flan T5

Google just released new SOTA LLMs on huggingface (better than OPT and Bloom). 
It's instruction-finetuned via reinforcement learning. This might be the best open-source LLM for our system, and should replace OPT.

* Paper: https://arxiv.org/abs/2210.11416
* huggingface: https://huggingface.co/docs/transformers/model_doc/flan-t5

In [1]:
!pip install accelerate sentencepiece

In [2]:
# pip install accelerate sentencepiece
import torch
import accelerate
from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto", torch_dtype=torch.float16)

/home/kastan/utils/miniconda3/envs/general_has_everything/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
CONTEXT = "{Example: A Two-Bit Gray Code Counter}Let's begin with a two-bit Gray code counter with no inputs.As we mentioned in Notes Set 2.1, a Gray code is a cycle over allbit patterns of a certain length in which consecutive patterns differin exactly one bit.For simplicity, our first few examples are based on counters anduse the internal stateof the FSM as the output values.  You should already knowhow to design combinational logic for the outputs if it were necessary.The inputs to a counter, if any, are typically limited to functionssuch as starting and stopping the counter, controlling the counting direction, and resetting the counter to a particular state.A fully-specified transition diagram for a two-bit Gray code counter appears below.With no inputs, the states simply form a loop, withthe counter moving from one state to the next each cycle.Each state in the diagram is marked with the internal state value S_1S_0 (before the ``/'') and the output Z_1Z_0 (after the ``/''), which are always equal for this counter.Based on the transition diagram, we can fill in the K-maps for the next-state values S_1^+ and S_0^+ as shown to the right of thetransition diagram, then derive algebraic expressions in the usual way to obtainS_1^+=S_0 and S_0^+={{S_1}}.We then use the next-state logic to develop the implementationshown on the far right, completing our first counter design."
PROMPT = "Please answer this person's question accurately, clearly and concicely. Context: "
QUESTION = "Question: What are the inputs and outputs of a Gray code counter? "
input_text = PROMPT + CONTEXT + QUESTION + "Answer: "

article = "UN Offizier sagt, dass weiter verhandelt werden muss in Syrien."
summary = "Weiter Verhandlung in Syrien."

input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
labels = tokenizer(text_target=summary, return_tensors="pt")

outputs = model.generate(input_ids, max_length=1024)
print(tokenizer.decode(outputs[0]))

<pad> The inputs to a counter, if any, are typically limited to functionssuch as starting and stopping the counter, controlling the counting direction, and resetting the counter to a particular state.</s>


In [3]:
# from transformers import T5Tokenizer, T5EncoderModel
# tokenizer = T5Tokenizer.from_pretrained("t5-small")
# model = T5EncoderModel.from_pretrained("t5-small")

input_ids = tokenizer(
    "Studies have been shown that owning a dog is good for you", return_tensors="pt"
).input_ids  # Batch size 1
outputs = model(input_ids=input_ids, )
last_hidden_states = outputs.last_hidden_state
last_hidden_states

ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [ ]:
last_hidden_states.shape

In [5]:
from transformers import T5Tokenizer, T5EncoderModel

tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5EncoderModel.from_pretrained("t5-small")
input_ids = tokenizer(
    "Studies have been shown that owning a dog is good for you", return_tensors="pt"
).input_ids  # Batch size 1
outputs = model(input_ids=input_ids)
last_hidden_states = outputs.last_hidden_state

Some weights of the model checkpoint at t5-small were not used when initializing T5EncoderModel: ['decoder.block.0.layer.1.EncDecAttention.o.weight', 'decoder.block.3.layer.0.SelfAttention.q.weight', 'decoder.block.1.layer.1.layer_norm.weight', 'decoder.block.1.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight', 'decoder.block.2.layer.2.DenseReluDense.wi.weight', 'decoder.block.1.layer.2.layer_norm.weight', 'decoder.block.5.layer.1.EncDecAttention.o.weight', 'decoder.block.4.layer.0.SelfAttention.q.weight', 'decoder.block.1.layer.0.layer_norm.weight', 'decoder.block.5.layer.0.SelfAttention.k.weight', 'decoder.block.3.layer.0.layer_norm.weight', 'decoder.block.4.layer.2.layer_norm.weight', 'decoder.block.5.layer.2.layer_norm.weight', 'decoder.block.2.layer.2.layer_norm.weight', 'decoder.block.0.layer.2.DenseReluDense.wi.weight', 'decoder.block.2.layer.0.SelfAttention.q.weight', 'decoder.block.0.layer.2.DenseReluDense.wo.weight', 'de

In [7]:
last_hidden_states.shape

torch.Size([1, 15, 512])